In [2]:
import sys, os

!{sys.executable} -m pip install -q pyvista planetary-computer pystac-client pystac stackstac

# specify GMTSAR installation path
PATH = os.environ['PATH']
if PATH.find('GMTSAR') == -1:
    PATH = os.environ['PATH'] + ':/usr/local/GMTSAR/bin/'
    %env PATH {PATH}

In [3]:
import xarray as xr
import numpy as np

import shutil
import pandas as pd
import geopandas as gpd
import json

from dask.distributed import Client
import dask


# plotting modules
import pyvista as pv
# magic trick for white background
pv.set_plot_theme("document")
import panel
panel.extension(comms='ipywidgets')
panel.extension('vtk')
from contextlib import contextmanager
import matplotlib.pyplot as plt


@contextmanager
def mpl_settings(settings):
    original_settings = {k: plt.rcParams[k] for k in settings}
    plt.rcParams.update(settings)
    yield
    plt.rcParams.update(original_settings)


plt.rcParams['figure.figsize'] = [12, 4]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['figure.titlesize'] = 24
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12


# define Pandas display settings
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 100)

from pygmtsar import S1, Stack, tqdm_dask, ASF, Tiles, XYZTiles

In [19]:
def clear_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # Remove file
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # Remove directory
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')

In [40]:
SUBSWATH = 2
FILENAMES = [
    'Sep 2015 - Dec 2015 Descending',
    'Aug 2018 - Sep 2018',
    'Oct 2020 - Nov 2020 Ascending',
    'Jul 2024 - Oct 2024 Descending'
]
FILENAME = FILENAMES[0]

def get_sar_1_collections_from(file_path):
    with open (file_path, 'r') as f:
        scenes = f.readlines()
    return [f.strip() for f in scenes]

SAR_1_COLLECTIONS_FILE = f'data/bursts/{FILENAME}.txt'
scenes = get_sar_1_collections_from(SAR_1_COLLECTIONS_FILE)
SCENES = scenes
SCENES.reverse()

WORKDIR = 'raw_golden'
DATADIR = 'data_golden'
DEM = f'{DATADIR}/dem.nc'

clear_directory(WORKDIR)
clear_directory(DATADIR)

geojson = '''
{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [121.0017, 14.5361]
  },
  "properties": {}
}
'''
AOI = gpd.GeoDataFrame.from_features([json.loads(geojson)])
AOI = AOI.buffer(0.02)

# Set these variables to None and you will be prompted to enter your username and password below.
asf_username = 'mirasnickanthony'
asf_password = 'e44 4E6 E447E S56E!'
asf = ASF(asf_username, asf_password)
print(asf.download(DATADIR, SCENES))
S1.download_orbits(DATADIR, S1.scan_slc(DATADIR))
Tiles().download_dem(AOI, filename=DEM, skip_exist=False)

# simple Dask initialization
if 'client' in globals():
    client.close()

client = Client()

ASF Downloading Bursts Catalog:   0%|          | 0/1 [00:00<?, ?it/s]

ASF Downloading Sentinel-1 Bursts:   0%|          | 0/54 [00:00<?, ?it/s]

ERROR: download attempt 1 failed for {
  "geometry": {
    "coordinates": [
      [
        [
          121.330072,
          14.21612
        ],
        [
          121.367222,
          14.400397
        ],
        [
          120.943933,
          14.48356
        ],
        [
          120.544148,
          14.561471
        ],
        [
          120.506034,
          14.369058
        ],
        [
          120.906302,
          14.295215
        ],
        [
          121.330072,
          14.21612
        ]
      ]
    ],
    "type": "Polygon"
  },
  "properties": {
    "additionalUrls": [
      "https://sentinel1-burst.asf.alaska.edu/S1A_IW_SLC__1SDV_20151213T214623_20151213T214650_009029_00CF3B_5DF0/IW2/VV/6.xml"
    ],
    "beamModeType": "IW",
    "browse": null,
    "burst": {
      "absoluteBurstID": 19392484,
      "azimuthAnxTime": "2733.5761637696",
      "azimuthTime": "2015-12-13T21:46:39.954141",
      "burstIndex": 6,
      "fullBurstID": "032_067577_IW2",
      "r

Tiles Parallel Downloading:   0%|          | 0/2 [00:00<?, ?it/s]

2024-11-09 15:22:01,533 - distributed.scheduler - WARNING - Detected different `run_spec` for key ('getitem-6514d150219ea6eb23d8b72da1da1646', 8, 0, 0) between two consecutive calls to `update_graph`. This can cause failures and deadlocks down the line. Please ensure unique key names. If you are using a standard dask collections, consider releasing all the data before resubmitting another computation. More details and help can be found at https://github.com/dask/dask/issues/9888. 
Debugging information
---------------------
old task state: waiting
old run_spec: (<function getitem at 0x7f1bac851f80>, (('where-17022da162be3f7c27982612e8cda067', 5, 0, 0), (array([0]), slice(None, None, None), slice(None, None, None))), {})
new run_spec: (<function execute_task at 0x7f1bd331ede0>, (('where-getitem-6514d150219ea6eb23d8b72da1da1646', 8, 0, 0),), {})
old token: ('tuple', [('3a54a0fe35af534a', []), ('tuple', [('tuple', ['where-17022da162be3f7c27982612e8cda067', 5, 0, 0]), ('tuple', [('942c203a

In [46]:
POLARIZATION = 'VH'
scenes = S1.scan_slc(DATADIR, subswath=SUBSWATH, polarization=POLARIZATION)
sbas = Stack(WORKDIR, drop_if_exists=True).set_scenes(scenes)
sbas.compute_reframe(AOI)
sbas.load_dem(DEM, AOI)
sbas.compute_align()
sbas.compute_geocode(1)
baseline_pairs = sbas.sbas_pairs(days=24)

NOTE: Found multiple scenes for a single day, use function Stack.reframe() to stitch the scenes
NOTE: auto set reference scene 2015-09-08. You can change it like Stack.set_reference("2022-01-20")


Reframing:   0%|          | 0/9 [00:00<?, ?it/s]

Save DEM on WGS84 Ellipsoid:   0%|          | 0/9000000.0 [00:00<?, ?it/s]

Aligning Reference:   0%|          | 0/1 [00:00<?, ?it/s]

Aligning Repeat:   0%|          | 0/8 [00:00<?, ?it/s]

...

Radar Transform Computing:   0%|          | 0/1 [00:00<?, ?it/s]

Radar Transform Saving:   0%|          | 0/9000000.0 [00:00<?, ?it/s]

Radar Transform Indexing:   0%|          | 0/9000000.0 [00:00<?, ?it/s]

Radar Inverse Transform Computing:   0%|          | 0/9000000.0 [00:00<?, ?it/s]

Satellite Look Vector Computing:   0%|          | 0/9000000.0 [00:00<?, ?it/s]

In [47]:
baseline_pairs

,ref,rep,ref_baseline,rep_baseline,pair,baseline,duration,rel
0,2015-09-08,2015-09-20,-0.00,-12.12,2015-09-08 2015-09-20,-12.12,12,NaT
1,2015-09-08,2015-10-02,-0.00,-7.67,2015-09-08 2015-10-02,-7.67,24,NaT
2,2015-09-20,2015-10-02,-12.12,-7.67,2015-09-20 2015-10-02,4.45,12,NaT
3,2015-09-20,2015-10-14,-12.12,11.42,2015-09-20 2015-10-14,23.54,24,NaT
4,2015-10-02,2015-10-14,-7.67,11.42,2015-10-02 2015-10-14,19.09,12,NaT
5,2015-10-02,2015-10-26,-7.67,-24.44,2015-10-02 2015-10-26,-16.77,24,NaT
6,2015-10-14,2015-10-26,11.42,-24.44,2015-10-14 2015-10-26,-35.86,12,NaT
7,2015-10-26,2015-11-19,-24.44,-5.64,2015-10-26 2015-11-19,18.80,24,NaT
8,2015-11-19,2015-12-01,-5.64,31.37,2015-11-19 2015-12-01,37.01,12,NaT
9,2015-11-19,2015-12-13,-5.64,77.26,2015-11-19 2015-12-13,82.90,24,NaT


In [48]:
sbas.set_landmask(None)
sbas.load_landmask('recurrence_120E_20Nv1_4_2021.tif')
landmask = (sbas.get_landmask()*-1)>-0.02
sbas.set_landmask(None)
landmask_ra = sbas.ll2ra(landmask)

In [49]:
### ONE SHOT
WAVELENGTH = 20
COARSEN_GRID = (1, 4)
sbas.compute_interferogram_multilook(
    baseline_pairs,
    'intf_mlook',
    wavelength=WAVELENGTH,
    phase=sbas.phasediff(baseline_pairs, sbas.open_data(), sbas.get_topo()),
    coarsen=COARSEN_GRID
)


ds_sbas = sbas.open_stack('intf_mlook')
ds_sbas = ds_sbas.where(landmask_ra.interp_like(ds_sbas, method='nearest'))
intf_sbas = ds_sbas.phase
corr_sbas = ds_sbas.correlation
sbas_phase_goldstein = sbas.goldstein(intf_sbas, corr_sbas, 8)
intf15m = sbas.interferogram(sbas_phase_goldstein)


tqdm_dask(result := dask.persist(corr_sbas, intf15m), desc='Compute Phase and Correlation')
corr, intf = result
corr_ll = sbas.ra2ll(corr_sbas)

Saving Interferogram 01...13 from 13:   0%|          | 0/9000000.0 [00:00<?, ?it/s]

Compute Phase and Correlation:   0%|          | 0/9000000.0 [00:00<?, ?it/s]

In [50]:
PRE_FLOOD_DOI = '2015-11-19 2015-12-01'
CO_FLOOD_DOI = '2015-12-01 2015-12-13'
corr_sbas_df = corr_ll.to_dataframe()
pre_flood_df = corr_sbas_df[corr_sbas_df.index.get_level_values(0) == PRE_FLOOD_DOI]
co_flood_df = corr_sbas_df[corr_sbas_df.index.get_level_values(0) == CO_FLOOD_DOI]
pre_flood_df.to_csv(f'csv/{PRE_FLOOD_DOI} {POLARIZATION}.csv')
co_flood_df.to_csv(f'csv/{CO_FLOOD_DOI} {POLARIZATION}.csv')
print('Saved Dataframes')

Saved Dataframes


# Conclusion